In [2]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as pyplot 
import json
import ast
import sklearn
import pyarrow
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.metrics.pairwise        import cosine_similarity
from sklearn.metrics.pairwise        import linear_kernel
from sklearn.feature_extraction.text import TfidfVectorizer


# Define the format of your dataset


[nltk_data] Downloading package vader_lexicon to C:\Users\D
[nltk_data]     Ryzen\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


## LECTURA DE LOS ARCHIVOS

In [ ]:
from surprise import Dataset, Reader
from surprise import SVD
from surprise.model_selection import cross_validate

In [9]:
df = pd.read_json('output_steam_games.json', orient='records', lines=True, convert_dates=True)
df.tail()

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
120440,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",1.99,0.0,773640.0,"Nikita ""Ghost_RUS"""
120441,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.99,0.0,733530.0,Sacada
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,0.0,610660.0,Laush Dmitriy Sergeevich
120443,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.99,0.0,658870.0,"xropi,stev3ns"
120444,None,None,Maze Run VR,None,http://store.steampowered.com/app/681550/Maze_...,None,"[Early Access, Adventure, Indie, Action, Simul...",http://steamcommunity.com/app/681550/reviews/?...,"[Single-player, Stats, Steam Leaderboards, HTC...",4.99,1.0,681550.0,None


In [23]:
data_list = []
file_path = 'australian_users_items.json'

#Abrir el archivo y procesar cada línea
with open(file_path, 'r', encoding='utf-8') as file:
    for line in file:
        try:
            # Usar ast.literal_eval para convertir la línea en un diccionario
            json_data = ast.literal_eval(line)
            data_list.append(json_data)
        except ValueError as e:
            print(f"Error en la línea: {line}")
            continue

#Crear un DataFrame a partir de la lista de diccionarios
df2 = pd.DataFrame(data_list)

In [4]:
df2.head()

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."


In [77]:
data_list = []
file_path = 'australian_user_reviews.json'

#Abrir el archivo y procesar cada línea
with open(file_path, 'r', encoding='utf-8') as file:
    for line in file:
        try:
            # Usar ast.literal_eval para convertir la línea en un diccionario
            json_data = ast.literal_eval(line)
            data_list.append(json_data)
        except ValueError as e:
            print(f"Error en la línea: {line}")
            continue

#Crear un DataFrame a partir de la lista de diccionarios
df3 = pd.DataFrame(data_list)

In [6]:
df3.head()

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."


# PREPARANDO EL PRIMER ARCHIVO

In [10]:
df.rename(columns = {'id':'item_id'}, inplace = True)


In [11]:
df1 = df.drop(columns=['app_name', 'url', 'reviews_url', 'specs', 'early_access'], axis=1)

In [83]:
df1["genres"] = df1["genres"].astype(str)

In [84]:
df1['genres'].str.strip("[]").str.replace("'", "")

0                                        None
1                                        None
2                                        None
3                                        None
4                                        None
                         ...                 
120440    Casual, Indie, Simulation, Strategy
120441                Casual, Indie, Strategy
120442              Indie, Racing, Simulation
120443                          Casual, Indie
120444                                   None
Name: genres, Length: 120445, dtype: object

In [86]:
df1 = df1.dropna()
df1

,publisher,genres,title,release_date,tags,price,item_id,developer
88310,Kotoshiro,"['Action', 'Casual', 'Indie', 'Simulation', 'S...",Lost Summoner Kitty,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",4.99,761140.0,Kotoshiro
88311,"Making Fun, Inc.","['Free to Play', 'Indie', 'RPG', 'Strategy']",Ironbound,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",Free To Play,643980.0,Secret Level SRL
88312,Poolians.com,"['Casual', 'Free to Play', 'Indie', 'Simulatio...",Real Pool 3D - Poolians,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",Free to Play,670290.0,Poolians.com
88313,彼岸领域,"['Action', 'Adventure', 'Casual']",弹炸人2222,2017-12-07,"[Action, Adventure, Casual]",0.99,767400.0,彼岸领域
88315,Trickjump Games Ltd,"['Action', 'Adventure', 'Simulation']",Battle Royale Trainer,2018-01-04,"[Action, Adventure, Simulation, FPS, Shooter, ...",3.99,772540.0,Trickjump Games Ltd
...,...,...,...,...,...,...,...,...
120439,Bidoniera Games,"['Action', 'Adventure', 'Casual', 'Indie']",Kebab it Up!,2018-01-04,"[Action, Indie, Casual, Violent, Adventure]",1.99,745400.0,Bidoniera Games
120440,Ghost_RUS Games,"['Casual', 'Indie', 'Simulation', 'Strategy']",Colony On Mars,2018-01-04,"[Strategy, Indie, Casual, Simulation]",1.99,773640.0,"Nikita ""Ghost_RUS"""
120441,Sacada,"['Casual', 'Indie', 'Strategy']",LOGistICAL: South Africa,2018-01-04,"[Strategy, Indie, Casual]",4.99,733530.0,Sacada
120442,Laush Studio,"['Indie', 'Racing', 'Simulation']",Russian Roads,2018-01-04,"[Indie, Simulation, Racing]",1.99,610660.0,Laush Dmitriy Sergeevich


In [16]:
def convertir_a_entero_o_cero(df1):
    try:
        return float(df1)
    except (ValueError, TypeError):
        return 0

# Aplicar la función a la columna 'columna_strings' del DataFrame
df1['price'] = df1['price'].apply(convertir_a_entero_o_cero)

In [17]:
df1

,publisher,genres,title,release_date,tags,price,item_id,developer
0,None,None,None,None,None,0.00,NaN,None
1,None,None,None,None,None,0.00,NaN,None
2,None,None,None,None,None,0.00,NaN,None
3,None,None,None,None,None,0.00,NaN,None
4,None,None,None,None,None,0.00,NaN,None
...,...,...,...,...,...,...,...,...
120440,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,2018-01-04,"[Strategy, Indie, Casual, Simulation]",1.99,773640.0,"Nikita ""Ghost_RUS"""
120441,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,2018-01-04,"[Strategy, Indie, Casual]",4.99,733530.0,Sacada
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,2018-01-04,"[Indie, Simulation, Racing]",1.99,610660.0,Laush Dmitriy Sergeevich
120443,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",4.99,658870.0,"xropi,stev3ns"


In [18]:
df1["item_id"] = df1["item_id"].astype(float)


In [19]:
df1["price"] = df1["price"].astype(float)

In [20]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 120445 entries, 0 to 120444
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   publisher     24083 non-null   object 
 1   genres        28852 non-null   object 
 2   title         30085 non-null   object 
 3   release_date  30068 non-null   object 
 4   tags          31972 non-null   object 
 5   price         120445 non-null  float64
 6   item_id       32133 non-null   float64
 7   developer     28836 non-null   object 
dtypes: float64(2), object(6)
memory usage: 8.3+ MB


In [21]:
df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce')

In [ ]:
df.to_parquet('I_items.parquet', index=False)

# Preparacion del 2 archivo

In [16]:
df2.columns

Index(['user_id', 'items_count', 'steam_id', 'user_url', 'items'], dtype='object')

In [22]:
df2.shape

(88310, 5)

In [19]:
df2= df2.explode(['items'])
ddf2 = pd.json_normalize(df2['items']).set_index(df2['items'].index)
df02= pd.concat([df2, ddf2], axis=1)

In [20]:
df02.tail()

,user_id,items_count,steam_id,user_url,items,item_id,item_name,playtime_forever,playtime_2weeks
88308,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,"{'item_id': '373330', 'item_name': 'All Is Dus...",373330,All Is Dust,0.0,0.0
88308,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,"{'item_id': '388490', 'item_name': 'One Way To...",388490,One Way To Die: Steam Edition,3.0,3.0
88308,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,"{'item_id': '521570', 'item_name': 'You Have 1...",521570,You Have 10 Seconds 2,4.0,4.0
88308,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,"{'item_id': '519140', 'item_name': 'Minds Eyes...",519140,Minds Eyes,3.0,3.0
88309,edward_tremethick,0,76561198331598578,http://steamcommunity.com/id/edward_tremethick,NaN,NaN,NaN,NaN,NaN


In [22]:
df02["item_id"]= df02["item_id"].astype(float)

NameError: name 'df02' is not defined

In [22]:
df02.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5170015 entries, 0 to 88309
Data columns (total 9 columns):
 #   Column            Dtype  
---  ------            -----  
 0   user_id           object 
 1   items_count       int64  
 2   steam_id          object 
 3   user_url          object 
 4   items             object 
 5   item_id           float64
 6   item_name         object 
 7   playtime_forever  float64
 8   playtime_2weeks   float64
dtypes: float64(3), int64(1), object(5)
memory usage: 394.4+ MB


In [24]:
df02.to_parquet('u_items.parquet', index=False)

EXPORTACION DEL SEGUNDO ARCHIVO A CSV

In [23]:
#se exporta la tabla como archivo separa por comas de nombre 'user_items'
df02.to_parquet('i_items.csv', index=False)

# Preparacion del 3 archivo

In [25]:
df3= df3.explode(['reviews'])
ddf3 = pd.json_normalize(df3['reviews']).set_index(df3['reviews'].index)
df03= pd.concat([df3, ddf3], axis=1)

In [26]:
df03.tail()

,user_id,user_url,reviews,funny,posted,last_edited,item_id,helpful,recommend,review
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"{'funny': '', 'posted': 'Posted July 10.', 'la...",,Posted July 10.,,70,No ratings yet,True,a must have classic from steam definitely wort...
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"{'funny': '', 'posted': 'Posted July 8.', 'las...",,Posted July 8.,,362890,No ratings yet,True,this game is a perfect remake of the original ...
25798,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,"{'funny': '1 person found this review funny', ...",1 person found this review funny,Posted July 3.,,273110,1 of 2 people (50%) found this review helpful,True,had so much fun plaing this and collecting res...
25798,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,"{'funny': '', 'posted': 'Posted July 20.', 'la...",,Posted July 20.,,730,No ratings yet,True,:D
25798,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,"{'funny': '', 'posted': 'Posted July 2.', 'las...",,Posted July 2.,,440,No ratings yet,True,so much fun :D


ANALISIS DE SENTIMIENTOS

In [27]:
sia = SentimentIntensityAnalyzer()

# Función para asignar valores según la escala / Function to assign values according to the scale
def get_sentiment_score(text):
    if pd.isnull(text) or text == '':
        return 1  # Return neutral if it is empty or NaN
    elif isinstance(text, str):
        sentiment = sia.polarity_scores(text)
        compound_score = sentiment['compound']
        if compound_score >= -0.05:
            return 2  # Good score
        elif compound_score <= -0.05:
            return 0  # Bad score
        else:
            return 1
    else:
        return 1  # Return neutral for non-string values

CREACION DE COLUMNA DE ANALISIS DE SENTIMIENTO

In [28]:
df03['review'] = df03['review'].astype(str)

# Aplicar la función get_sentiment_score a la columna 'review' 
df03['sentiment_score'] = df03['review'].apply(get_sentiment_score)
df03


,user_id,user_url,reviews,funny,posted,last_edited,item_id,helpful,recommend,review,sentiment_score
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted November 5, 20...",,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,2
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted July 15, 2011....",,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.,2
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted April 21, 2011...",,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...,2
1,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'posted': 'Posted June 24, 2014....",,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,2
1,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'posted': 'Posted September 8, 2...",,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...,2
...,...,...,...,...,...,...,...,...,...,...,...
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"{'funny': '', 'posted': 'Posted July 10.', 'la...",,Posted July 10.,,70,No ratings yet,True,a must have classic from steam definitely wort...,2
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"{'funny': '', 'posted': 'Posted July 8.', 'las...",,Posted July 8.,,362890,No ratings yet,True,this game is a perfect remake of the original ...,2
25798,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,"{'funny': '1 person found this review funny', ...",1 person found this review funny,Posted July 3.,,273110,1 of 2 people (50%) found this review helpful,True,had so much fun plaing this and collecting res...,2
25798,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,"{'funny': '', 'posted': 'Posted July 20.', 'la...",,Posted July 20.,,730,No ratings yet,True,:D,2


CREACION DEL 3 ARCHIVO A PARQUET PARA QUE CONSUMA MENOS ESPACIO

In [30]:
df03.to_parquet('r_items.parquet', index=False)

# LECTURA DE LOS ARCHIVOS PARA UNIRLOS 

In [11]:
df2 = pd.read_parquet("u_items.parquet")
df2

,user_id,items_count,steam_id,user_url,items,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '10', 'item_name': 'Counter-Strike...",10.0,Counter-Strike,6.0,0.0
1,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '20', 'item_name': 'Team Fortress ...",20.0,Team Fortress Classic,0.0,0.0
2,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '30', 'item_name': 'Day of Defeat'...",30.0,Day of Defeat,7.0,0.0
3,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '40', 'item_name': 'Deathmatch Cla...",40.0,Deathmatch Classic,0.0,0.0
4,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '50', 'item_name': 'Half-Life: Opp...",50.0,Half-Life: Opposing Force,0.0,0.0
...,...,...,...,...,...,...,...,...,...
5170010,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,"{'item_id': '373330', 'item_name': 'All Is Dus...",373330.0,All Is Dust,0.0,0.0
5170011,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,"{'item_id': '388490', 'item_name': 'One Way To...",388490.0,One Way To Die: Steam Edition,3.0,3.0
5170012,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,"{'item_id': '521570', 'item_name': 'You Have 1...",521570.0,You Have 10 Seconds 2,4.0,4.0
5170013,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,"{'item_id': '519140', 'item_name': 'Minds Eyes...",519140.0,Minds Eyes,3.0,3.0


BORRANDO COLUMNAS INNESESARIAS

In [12]:
df2 = df2.drop(columns=['user_url', 'items', 'user_url'])

In [13]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5170015 entries, 0 to 5170014
Data columns (total 7 columns):
 #   Column            Dtype  
---  ------            -----  
 0   user_id           object 
 1   items_count       int64  
 2   steam_id          object 
 3   item_id           float64
 4   item_name         object 
 5   playtime_forever  float64
 6   playtime_2weeks   float64
dtypes: float64(3), int64(1), object(3)
memory usage: 276.1+ MB


LECTURA DEL 3 ARCHIVO QUE HABIAMOS CONVERTIDO A PARQUET

In [14]:
df3 = pd.read_parquet("r_items.parquet")
df3

,user_id,user_url,reviews,funny,posted,last_edited,item_id,helpful,recommend,review,sentiment_score
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'helpful': 'No ratings yet', 'it...",,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,2
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'helpful': 'No ratings yet', 'it...",,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.,2
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'helpful': 'No ratings yet', 'it...",,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...,2
3,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'helpful': '15 of 20 people (75%...",,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,2
4,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'helpful': '0 of 1 people (0%) f...",,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...,2
...,...,...,...,...,...,...,...,...,...,...,...
59328,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"{'funny': '', 'helpful': 'No ratings yet', 'it...",,Posted July 10.,,70,No ratings yet,True,a must have classic from steam definitely wort...,2
59329,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"{'funny': '', 'helpful': 'No ratings yet', 'it...",,Posted July 8.,,362890,No ratings yet,True,this game is a perfect remake of the original ...,2
59330,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,"{'funny': '1 person found this review funny', ...",1 person found this review funny,Posted July 3.,,273110,1 of 2 people (50%) found this review helpful,True,had so much fun plaing this and collecting res...,2
59331,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,"{'funny': '', 'helpful': 'No ratings yet', 'it...",,Posted July 20.,,730,No ratings yet,True,:D,2


In [15]:
df3 = df3.drop(columns=['user_url', 'funny', 'posted', 'helpful', 'last_edited'])

In [16]:
df3["item_id"] = df3["item_id"].astype(float)

In [17]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59333 entries, 0 to 59332
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   user_id          59333 non-null  object 
 1   reviews          59305 non-null  object 
 2   item_id          59305 non-null  float64
 3   recommend        59305 non-null  object 
 4   review           59333 non-null  object 
 5   sentiment_score  59333 non-null  int64  
dtypes: float64(1), int64(1), object(4)
memory usage: 2.7+ MB


# UNION DEL PRIMER Y SEGUNDO DF LEIDOS

In [18]:
df123= pd.merge(df2, df3, on='user_id')

MODIFICACION DE NOMBRE DE COLUMNAS PARA PODERLOS UNIR

In [19]:
df123.rename(columns = {'item_id_y':'item_id'}, inplace = True)

In [67]:
df123.rename(columns = {'item_id_x':'item_id'}, inplace = True)

In [76]:
df123

,user_id,items_count,item_id_x,item_name,playtime_forever,playtime_2weeks,item_id,recommend,sentiment_score
0,76561197970982479,277,10.0,Counter-Strike,6.0,0.0,1250.0,True,2
1,76561197970982479,277,10.0,Counter-Strike,6.0,0.0,22200.0,True,2
2,76561197970982479,277,10.0,Counter-Strike,6.0,0.0,43110.0,True,2
3,76561197970982479,277,20.0,Team Fortress Classic,0.0,0.0,1250.0,True,2
4,76561197970982479,277,20.0,Team Fortress Classic,0.0,0.0,22200.0,True,2
...,...,...,...,...,...,...,...,...,...
7817956,76561198312638244,36,379720.0,DOOM,1178.0,0.0,70.0,True,2
7817957,76561198312638244,36,379720.0,DOOM,1178.0,0.0,362890.0,True,2
7817958,LydiaMorley,0,NaN,None,NaN,NaN,273110.0,True,2
7817959,LydiaMorley,0,NaN,None,NaN,NaN,730.0,True,2


In [32]:
DataSet_Final = pd.merge(df123, df1, on='item_id')

CREACION DE UNA COLUMNA A FORMATO FECHA INDEPENDIENTE

In [34]:
DataSet_Final['anio'] = pd.to_datetime(DataSet_Final["release_date"], errors='coerce')

In [35]:
DataSet_Final

,user_id,items_count,steam_id,item_id_x,item_name,playtime_forever,playtime_2weeks,reviews,item_id,recommend,review,sentiment_score,publisher,genres,title,release_date,tags,price,developer,anio
0,76561197970982479,277,76561197970982479,10.0,Counter-Strike,6.0,0.0,"{'funny': '', 'helpful': 'No ratings yet', 'it...",1250.0,True,Simple yet with great replayability. In my opi...,2,Tripwire Interactive,[Action],Killing Floor,2009-05-14,"[FPS, Zombies, Co-op, Survival, Action, Multip...",19.99,Tripwire Interactive,2009-05-14
1,76561197970982479,277,76561197970982479,10.0,Counter-Strike,6.0,0.0,"{'funny': '', 'helpful': 'No ratings yet', 'it...",22200.0,True,It's unique and worth a playthrough.,2,ACE Team,"[Action, Indie]",Zeno Clash,2009-04-21,"[Action, Indie, Surreal, Beat 'em up, FPS, Fig...",9.99,ACE Team,2009-04-21
2,76561197970982479,277,76561197970982479,20.0,Team Fortress Classic,0.0,0.0,"{'funny': '', 'helpful': 'No ratings yet', 'it...",1250.0,True,Simple yet with great replayability. In my opi...,2,Tripwire Interactive,[Action],Killing Floor,2009-05-14,"[FPS, Zombies, Co-op, Survival, Action, Multip...",19.99,Tripwire Interactive,2009-05-14
3,76561197970982479,277,76561197970982479,20.0,Team Fortress Classic,0.0,0.0,"{'funny': '', 'helpful': 'No ratings yet', 'it...",22200.0,True,It's unique and worth a playthrough.,2,ACE Team,"[Action, Indie]",Zeno Clash,2009-04-21,"[Action, Indie, Surreal, Beat 'em up, FPS, Fig...",9.99,ACE Team,2009-04-21
4,76561197970982479,277,76561197970982479,30.0,Day of Defeat,7.0,0.0,"{'funny': '', 'helpful': 'No ratings yet', 'it...",1250.0,True,Simple yet with great replayability. In my opi...,2,Tripwire Interactive,[Action],Killing Floor,2009-05-14,"[FPS, Zombies, Co-op, Survival, Action, Multip...",19.99,Tripwire Interactive,2009-05-14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6452296,76561198312638244,36,76561198312638244,379720.0,DOOM,1178.0,0.0,"{'funny': '', 'helpful': 'No ratings yet', 'it...",70.0,True,a must have classic from steam definitely wort...,2,Valve,[Action],Half-Life,1998-11-08,"[FPS, Classic, Action, Sci-fi, Singleplayer, S...",9.99,Valve,1998-11-08
6452297,76561198312638244,36,76561198312638244,379720.0,DOOM,1178.0,0.0,"{'funny': '', 'helpful': 'No ratings yet', 'it...",362890.0,True,this game is a perfect remake of the original ...,2,Crowbar Collective,"[Action, Adventure, Indie, Early Access]",Black Mesa,2015-05-05,"[Early Access, FPS, Remake, Action, Sci-fi, Fi...",19.99,Crowbar Collective,2015-05-05
6452298,LydiaMorley,0,76561198313816521,NaN,None,NaN,NaN,"{'funny': '1 person found this review funny', ...",273110.0,True,had so much fun plaing this and collecting res...,2,Nexon Europe GmbH,"[Action, Casual, Free to Play, Massively Multi...",Counter-Strike Nexon: Zombies,2014-10-07,"[Free to Play, Zombies, Multiplayer, FPS, Shoo...",0.00,Nexon,2014-10-07
6452299,LydiaMorley,0,76561198313816521,NaN,None,NaN,NaN,"{'funny': '', 'helpful': 'No ratings yet', 'it...",730.0,True,:D,2,Valve,[Action],Counter-Strike: Global Offensive,2012-08-21,"[FPS, Multiplayer, Shooter, Action, Team-Based...",14.99,Valve,2012-08-21


In [36]:
DataSet_Final["developer"] = DataSet_Final["developer"].astype(str)

OBSERVAR TIPOS DE DATOS DESEADOS

In [184]:
DataSet_Final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6452301 entries, 0 to 6452300
Data columns (total 16 columns):
 #   Column            Dtype         
---  ------            -----         
 0   user_id           object        
 1   items_count       int64         
 2   item_id_x         float64       
 3   item_name         object        
 4   playtime_forever  float64       
 5   playtime_2weeks   float64       
 6   item_id           float64       
 7   recommend         object        
 8   sentiment_score   int64         
 9   publisher         object        
 10  genres            object        
 11  title             object        
 12  release_date      object        
 13  price             float64       
 14  developer         object        
 15  anio              datetime64[ns]
dtypes: datetime64[ns](1), float64(5), int64(2), object(8)
memory usage: 787.6+ MB


LECTURA DE ARCHIVO FINAL PARA NO VOLVER A EJECUTAR EL CODIGO

In [4]:
DataSet_Final = pd.read_parquet("DataSet_Final.parquet", engine='pyarrow')

In [5]:
DataSet_Final["genres"] = DataSet_Final["genres"].astype(str)

In [6]:
DataSet_Final['genres'] = DataSet_Final['genres'].str.strip("[]").str.replace("'", "")

In [7]:
DataSet_Final.reset_index(drop=True)

,user_id,items_count,steam_id,item_id_x,item_name,playtime_forever,playtime_2weeks,reviews,item_id,recommend,review,sentiment_score,publisher,genres,title,release_date,tags,price,developer,anio
0,76561197970982479,277,76561197970982479,10.0,Counter-Strike,6.0,0.0,"{'funny': '', 'helpful': 'No ratings yet', 'it...",1250.0,True,Simple yet with great replayability. In my opi...,2,Tripwire Interactive,Action,Killing Floor,2009-05-14,"[FPS, Zombies, Co-op, Survival, Action, Multip...",19.99,Tripwire Interactive,2009-05-14
1,76561197970982479,277,76561197970982479,10.0,Counter-Strike,6.0,0.0,"{'funny': '', 'helpful': 'No ratings yet', 'it...",22200.0,True,It's unique and worth a playthrough.,2,ACE Team,Action Indie,Zeno Clash,2009-04-21,"[Action, Indie, Surreal, Beat 'em up, FPS, Fig...",9.99,ACE Team,2009-04-21
2,76561197970982479,277,76561197970982479,20.0,Team Fortress Classic,0.0,0.0,"{'funny': '', 'helpful': 'No ratings yet', 'it...",1250.0,True,Simple yet with great replayability. In my opi...,2,Tripwire Interactive,Action,Killing Floor,2009-05-14,"[FPS, Zombies, Co-op, Survival, Action, Multip...",19.99,Tripwire Interactive,2009-05-14
3,76561197970982479,277,76561197970982479,20.0,Team Fortress Classic,0.0,0.0,"{'funny': '', 'helpful': 'No ratings yet', 'it...",22200.0,True,It's unique and worth a playthrough.,2,ACE Team,Action Indie,Zeno Clash,2009-04-21,"[Action, Indie, Surreal, Beat 'em up, FPS, Fig...",9.99,ACE Team,2009-04-21
4,76561197970982479,277,76561197970982479,30.0,Day of Defeat,7.0,0.0,"{'funny': '', 'helpful': 'No ratings yet', 'it...",1250.0,True,Simple yet with great replayability. In my opi...,2,Tripwire Interactive,Action,Killing Floor,2009-05-14,"[FPS, Zombies, Co-op, Survival, Action, Multip...",19.99,Tripwire Interactive,2009-05-14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6452296,76561198312638244,36,76561198312638244,379720.0,DOOM,1178.0,0.0,"{'funny': '', 'helpful': 'No ratings yet', 'it...",70.0,True,a must have classic from steam definitely wort...,2,Valve,Action,Half-Life,1998-11-08,"[FPS, Classic, Action, Sci-fi, Singleplayer, S...",9.99,Valve,1998-11-08
6452297,76561198312638244,36,76561198312638244,379720.0,DOOM,1178.0,0.0,"{'funny': '', 'helpful': 'No ratings yet', 'it...",362890.0,True,this game is a perfect remake of the original ...,2,Crowbar Collective,Action Adventure Indie Early Access,Black Mesa,2015-05-05,"[Early Access, FPS, Remake, Action, Sci-fi, Fi...",19.99,Crowbar Collective,2015-05-05
6452298,LydiaMorley,0,76561198313816521,NaN,None,NaN,NaN,"{'funny': '1 person found this review funny', ...",273110.0,True,had so much fun plaing this and collecting res...,2,Nexon Europe GmbH,Action Casual Free to Play Massively Multiplayer,Counter-Strike Nexon: Zombies,2014-10-07,"[Free to Play, Zombies, Multiplayer, FPS, Shoo...",0.00,Nexon,2014-10-07
6452299,LydiaMorley,0,76561198313816521,NaN,None,NaN,NaN,"{'funny': '', 'helpful': 'No ratings yet', 'it...",730.0,True,:D,2,Valve,Action,Counter-Strike: Global Offensive,2012-08-21,"[FPS, Multiplayer, Shooter, Action, Team-Based...",14.99,Valve,2012-08-21


In [110]:
DataSet_Final["genres"].unique()

array(['Action', 'Action Indie', 'Indie Simulation', 'Adventure Indie',
       'Action Indie RPG', 'Action Indie RPG Strategy',
       'Indie Simulation Strategy', 'Adventure', 'RPG',
       'Action Free to Play Massively Multiplayer RPG',
       'Action Adventure', 'Action RPG',
       'Action Adventure Casual Indie RPG',
       'Action Free to Play Indie Strategy', 'Action Free to Play',
       'Action Indie Racing Sports',
       'Action Adventure Casual Free to Play Indie',
       'Action Free to Play Indie', 'Action Indie Simulation',
       'Action Simulation Strategy',
       'Action Indie Massively Multiplayer Early Access',
       'Adventure Casual Free to Play Indie',
       'Action Adventure Free to Play Massively Multiplayer RPG',
       'Action Adventure Casual Indie', 'Indie',
       'Action Adventure Indie RPG', 'Action Adventure RPG',
       'Action Adventure Indie',
       'Action Free to Play Massively Multiplayer Strategy',
       'Adventure Indie Simulation', 'Free 

In [104]:
DataSet_Final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6452301 entries, 0 to 6452300
Data columns (total 20 columns):
 #   Column            Dtype         
---  ------            -----         
 0   user_id           object        
 1   items_count       int64         
 2   steam_id          object        
 3   item_id_x         float64       
 4   item_name         object        
 5   playtime_forever  float64       
 6   playtime_2weeks   float64       
 7   reviews           object        
 8   item_id           float64       
 9   recommend         bool          
 10  review            object        
 11  sentiment_score   int64         
 12  publisher         object        
 13  genres            object        
 14  title             object        
 15  release_date      object        
 16  tags              object        
 17  price             float64       
 18  developer         object        
 19  anio              datetime64[ns]
dtypes: bool(1), datetime64[ns](1), float64(5), int

CREACION DEL DATASET FINAL

In [ ]:
DataSet_Final.to_parquet("DataSet_Final.parquet")

# QUITAR PESO PARA SUBIR A GITHUB

In [10]:
df = pd.read_parquet("DataSet_Final.parquet")

In [4]:
df

,user_id,items_count,steam_id,item_id_x,item_name,playtime_forever,playtime_2weeks,reviews,item_id,recommend,review,sentiment_score,publisher,genres,title,release_date,tags,price,developer,anio
0,76561197970982479,277,76561197970982479,10.0,Counter-Strike,6.0,0.0,"{'funny': '', 'helpful': 'No ratings yet', 'it...",1250.0,True,Simple yet with great replayability. In my opi...,2,Tripwire Interactive,Action,Killing Floor,2009-05-14,"[FPS, Zombies, Co-op, Survival, Action, Multip...",19.99,Tripwire Interactive,2009-05-14
1,76561197970982479,277,76561197970982479,10.0,Counter-Strike,6.0,0.0,"{'funny': '', 'helpful': 'No ratings yet', 'it...",22200.0,True,It's unique and worth a playthrough.,2,ACE Team,Action Indie,Zeno Clash,2009-04-21,"[Action, Indie, Surreal, Beat 'em up, FPS, Fig...",9.99,ACE Team,2009-04-21
2,76561197970982479,277,76561197970982479,20.0,Team Fortress Classic,0.0,0.0,"{'funny': '', 'helpful': 'No ratings yet', 'it...",1250.0,True,Simple yet with great replayability. In my opi...,2,Tripwire Interactive,Action,Killing Floor,2009-05-14,"[FPS, Zombies, Co-op, Survival, Action, Multip...",19.99,Tripwire Interactive,2009-05-14
3,76561197970982479,277,76561197970982479,20.0,Team Fortress Classic,0.0,0.0,"{'funny': '', 'helpful': 'No ratings yet', 'it...",22200.0,True,It's unique and worth a playthrough.,2,ACE Team,Action Indie,Zeno Clash,2009-04-21,"[Action, Indie, Surreal, Beat 'em up, FPS, Fig...",9.99,ACE Team,2009-04-21
4,76561197970982479,277,76561197970982479,30.0,Day of Defeat,7.0,0.0,"{'funny': '', 'helpful': 'No ratings yet', 'it...",1250.0,True,Simple yet with great replayability. In my opi...,2,Tripwire Interactive,Action,Killing Floor,2009-05-14,"[FPS, Zombies, Co-op, Survival, Action, Multip...",19.99,Tripwire Interactive,2009-05-14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6452296,76561198312638244,36,76561198312638244,379720.0,DOOM,1178.0,0.0,"{'funny': '', 'helpful': 'No ratings yet', 'it...",70.0,True,a must have classic from steam definitely wort...,2,Valve,Action,Half-Life,1998-11-08,"[FPS, Classic, Action, Sci-fi, Singleplayer, S...",9.99,Valve,1998-11-08
6452297,76561198312638244,36,76561198312638244,379720.0,DOOM,1178.0,0.0,"{'funny': '', 'helpful': 'No ratings yet', 'it...",362890.0,True,this game is a perfect remake of the original ...,2,Crowbar Collective,Action Adventure Indie Early Access,Black Mesa,2015-05-05,"[Early Access, FPS, Remake, Action, Sci-fi, Fi...",19.99,Crowbar Collective,2015-05-05
6452298,LydiaMorley,0,76561198313816521,NaN,None,NaN,NaN,"{'funny': '1 person found this review funny', ...",273110.0,True,had so much fun plaing this and collecting res...,2,Nexon Europe GmbH,Action Casual Free to Play Massively Multiplayer,Counter-Strike Nexon: Zombies,2014-10-07,"[Free to Play, Zombies, Multiplayer, FPS, Shoo...",0.00,Nexon,2014-10-07
6452299,LydiaMorley,0,76561198313816521,NaN,None,NaN,NaN,"{'funny': '', 'helpful': 'No ratings yet', 'it...",730.0,True,:D,2,Valve,Action,Counter-Strike: Global Offensive,2012-08-21,"[FPS, Multiplayer, Shooter, Action, Team-Based...",14.99,Valve,2012-08-21


In [11]:
df = df.drop(columns=['reviews', 'review', 'tags', 'release_date'])
df

,user_id,items_count,steam_id,item_id_x,item_name,playtime_forever,playtime_2weeks,item_id,recommend,sentiment_score,publisher,genres,title,price,developer,anio
0,76561197970982479,277,76561197970982479,10.0,Counter-Strike,6.0,0.0,1250.0,True,2,Tripwire Interactive,Action,Killing Floor,19.99,Tripwire Interactive,2009-05-14
1,76561197970982479,277,76561197970982479,10.0,Counter-Strike,6.0,0.0,22200.0,True,2,ACE Team,Action Indie,Zeno Clash,9.99,ACE Team,2009-04-21
2,76561197970982479,277,76561197970982479,20.0,Team Fortress Classic,0.0,0.0,1250.0,True,2,Tripwire Interactive,Action,Killing Floor,19.99,Tripwire Interactive,2009-05-14
3,76561197970982479,277,76561197970982479,20.0,Team Fortress Classic,0.0,0.0,22200.0,True,2,ACE Team,Action Indie,Zeno Clash,9.99,ACE Team,2009-04-21
4,76561197970982479,277,76561197970982479,30.0,Day of Defeat,7.0,0.0,1250.0,True,2,Tripwire Interactive,Action,Killing Floor,19.99,Tripwire Interactive,2009-05-14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6452296,76561198312638244,36,76561198312638244,379720.0,DOOM,1178.0,0.0,70.0,True,2,Valve,Action,Half-Life,9.99,Valve,1998-11-08
6452297,76561198312638244,36,76561198312638244,379720.0,DOOM,1178.0,0.0,362890.0,True,2,Crowbar Collective,Action Adventure Indie Early Access,Black Mesa,19.99,Crowbar Collective,2015-05-05
6452298,LydiaMorley,0,76561198313816521,NaN,None,NaN,NaN,273110.0,True,2,Nexon Europe GmbH,Action Casual Free to Play Massively Multiplayer,Counter-Strike Nexon: Zombies,0.00,Nexon,2014-10-07
6452299,LydiaMorley,0,76561198313816521,NaN,None,NaN,NaN,730.0,True,2,Valve,Action,Counter-Strike: Global Offensive,14.99,Valve,2012-08-21


In [12]:
df.to_parquet("DataSet_Final2.parquet")